In [1]:
#Import dependency libraries
import dependency libraries
from random import seed
from random import randrange, random
from random import 
from math import exp
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.datasets import load_iris

In [2]:
#Load the Dataset
iris = load_iris()
type(iris)

sklearn.utils.Bunch

In [3]:
# Names of features/columns in iris dataset
print(iris.feature_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [4]:
#shape of dataset
iris.data.shape

(150, 4)

In [5]:
#Function to formatting dataset
def data_set(data,target):
  data = data.tolist()
  for i in range(len(data)):
    data[i].extend([target[i]])
  return data

In [6]:
data = iris.data
target = iris.target

In [7]:
#formatting dataset
iris_dataset = data_set(data,target)
len(iris_dataset)

150

In [8]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = list(dataset)
        fold_size = int(len(dataset) / n_folds)
        for i in range(n_folds):
                fold = list()
                while len(fold) < fold_size:
                        index = randrange(len(dataset_copy))
                        fold.append(dataset_copy.pop(index))
                dataset_split.append(fold)
        return dataset_split

In [9]:
# Evaluate an algorithm using a cross validation split
def run_algorithm(dataset, algorithm, n_folds, *args):
        
        folds = cross_validation_split(dataset, n_folds)
        #for fold in folds:
                #print("Fold {} \n \n".format(fold))
        scores = list()
        Nfold = 1
        for fold in folds:
                #print("Test Fold {} \n \n".format(fold))
                train_set = list(folds)
                train_set.remove(fold)
                train_set = sum(train_set, [])
                test_set = list()
                for row in fold:
                        row_copy = list(row)
                        test_set.append(row_copy)
                        row_copy[-1] = None
                predicted = algorithm(train_set, test_set, *args)

                #Get the actual target values from test dataset
                actual = [row[-1] for row in fold]
                #Evaluation for test dataset
                accuracy = accuracy_met(actual, predicted)
                cm = confusion_matrix(actual, predicted)
                print('For Fold {} results:'.format(Nfold))
                print('Confusion Matrix:')
                print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in cm]))
                confusionmatrix = np.matrix(cm)
                FP = cm.sum(axis=0) - np.diag(cm)
                FN = cm.sum(axis=1) - np.diag(cm)
                TP = np.diag(cm)
                TN = cm.sum() - (FP + FN + TP)
                #print('False Positives\n {}'.format(FP))
                #print('False Negetives\n {}'.format(FN))
                #print('True Positives\n {}'.format(TP))
                #print('True Negetives\n {}'.format(TN))
                TPR = TP/(TP+FN)
                print('Sensitivity \n {}'.format(TPR))
                TNR = TN/(TN+FP)
                print('Specificity \n {}'.format(TNR))
                Precision = TP/(TP+FP)
                print('Precision \n {}'.format(Precision))
                Recall = TP/(TP+FN)
                print('Recall \n {}'.format(Recall))
                Acc = (TP+TN)/(TP+TN+FP+FN)
                #print('Áccuracy \n{}'.format(Acc))
                Fscore = 2*(Precision*Recall)/(Precision+Recall)
                print('FScore \n{}'.format(Fscore))
                scores.append(accuracy)
                Nfold+=1
                print('\n\n')
        return 'Different Folds Accuracy score:', scores

In [10]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
        n_inputs = len(train[0]) - 1
        n_outputs = len(set([row[-1] for row in train]))
        network = initialize_network(n_inputs, n_hidden, n_outputs)
        train_network(network, train, l_rate, n_epoch, n_outputs)
        #print("network {}\n".format(network))
        predictions = list()
        for row in test:
                prediction = predict(network, row)
                predictions.append(prediction)
        return(predictions)

In [11]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
        network = list()
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)],'prev':[0 for i in range(n_hidden+1)]} for i in range(n_outputs)]
        network.append(output_layer)
        #print(network)
        return network

In [12]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
        for epoch in range(n_epoch):
                for row in train:
                        outputs = forward_propagate(network, row)
                        #print(network)
                        expected = [0 for i in range(n_outputs)]
                        expected[row[-1]] = 1
                        #print("expected row{}\n".format(expected))
                        backward_propagate_error(network, expected)
                        update_weights(network, row, l_rate)

In [13]:
# Forward propagate input to a network output
def forward_propagate(network, row):
        inputs = row
        for layer in network:
                new_inputs = []
                for neuron in layer:
                        activation = activate(neuron['weights'], inputs)
                        neuron['output'] = transfer(activation)
                        new_inputs.append(neuron['output'])                       
                inputs = new_inputs
        return inputs

In [14]:
# Calculate neuron activation for an input
def activate(weights, inputs):
        activation = weights[-1]
        for i in range(len(weights)-1):
                activation += weights[i] * inputs[i]
        return activation

In [15]:
# Transfer neuron activation
def transfer(activation):
        return 1.0 / (1.0 + exp(-activation))

In [16]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
        return output * (1.0 - output)

In [17]:
# Make a prediction with a network
def predict(network, row):
        outputs = forward_propagate(network, row)
        return outputs.index(max(outputs))

In [18]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
        total_error = list()
        for i in reversed(range(len(network))):
                layer = network[i]
                errors = list()
                total_error = list()
                if i != len(network)-1:
                        for j in range(len(layer)):
                                error = 0.0
                                for neuron in network[i + 1]:
                                        error += (neuron['weights'][j] * neuron['delta'])
                                errors.append(error)
                else:
                        for j in range(len(layer)):
                                neuron = layer[j]
                                errors.append(expected[j] - neuron['output'])
                for j in range(len(layer)):
                        neuron = layer[j]
                        neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [19]:
# Update network weights with error
def update_weights(network, row, l_rate):
        for i in range(len(network)):
                inputs = row[:-1]                
                if i != 0:
                        inputs = [neuron['output'] for neuron in network[i - 1]]
                for neuron in network[i]:
                        for j in range(len(inputs)):
                                temp = l_rate * neuron['delta'] * inputs[j] + mu * neuron['prev'][j]
                                
                                neuron['weights'][j] += temp
                                #print("neuron weight{} \n".format(neuron['weights'][j]))
                                neuron['prev'][j] = temp
                        temp = l_rate * neuron['delta'] + mu * neuron['prev'][-1]
                        neuron['weights'][-1] += temp
                        neuron['prev'][-1] = temp

In [20]:
# Calculate accuracy percentage
def accuracy_met(actual, predicted):
        correct = 0
        for i in range(len(actual)):
                if actual[i] == predicted[i]:
                        correct += 1
        return correct / float(len(actual)) * 100.0

In [21]:
# Test Backprop on Seeds dataset
seed(1)

# evaluate algorithm
n_folds = 5
l_rate = 0.1
mu=0.001
n_epoch = 500
n_hidden = 5

In [22]:
scores = run_algorithm(iris_dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

For Fold 1 results:
Confusion Matrix:
  11   0   0
   0   7   1
   0   0  11
Sensitivity 
 [1.    0.875 1.   ]
Specificity 
 [1.         1.         0.94736842]
Precision 
 [1.         1.         0.91666667]
Recall 
 [1.    0.875 1.   ]
FScore 
[1.         0.93333333 0.95652174]



For Fold 2 results:
Confusion Matrix:
  10   0   0
   0   9   2
   0   0   9
Sensitivity 
 [1.         0.81818182 1.        ]
Specificity 
 [1.        1.        0.9047619]
Precision 
 [1.         1.         0.81818182]
Recall 
 [1.         0.81818182 1.        ]
FScore 
[1.  0.9 0.9]



For Fold 3 results:
Confusion Matrix:
   6   0   0
   0  10   0
   0   0  14
Sensitivity 
 [1. 1. 1.]
Specificity 
 [1. 1. 1.]
Precision 
 [1. 1. 1.]
Recall 
 [1. 1. 1.]
FScore 
[1. 1. 1.]



For Fold 4 results:
Confusion Matrix:
  10   0   0
   0   9   1
   0   0  10
Sensitivity 
 [1.  0.9 1. ]
Specificity 
 [1.   1.   0.95]
Precision 
 [1.         1.         0.90909091]
Recall 
 [1.  0.9 1. ]
FScore 
[1.         0.94736842 0

In [23]:
scores

('Different Folds Accuracy score:',
 [96.66666666666667,
  93.33333333333333,
  100.0,
  96.66666666666667,
  86.66666666666667])